## <span style="color:#ff5f27">📝 Imports </span>


In [82]:
import os
import hopsworks

## <span style="color:#ff5f27">🔮 Connect to Hopsworks </span>

In [83]:
import hopsworks

project = hopsworks.login()

# Connect to Hopsworks Model Registry
mr = project.get_model_registry()

dataset_api = project.get_dataset_api()

2025-04-17 19:20:56,387 INFO: Initializing external client
2025-04-17 19:20:56,391 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-17 19:20:56,992 WARNING: UserWarning: The installed hopsworks client version 4.1.8 may not be compatible with the connected Hopsworks backend version 4.2.0. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-04-17 19:20:57,962 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788


## <span style="color:#ff5f27">🚀 Ranking Model Deployment </span>


You start by deploying your ranking model. Since it is a CatBoost model you need to implement a `Predict` class that tells Hopsworks how to load the model and how to use it.


In [84]:
weather_ranking_model = mr.get_best_model(
    name="weather_ranking_model", 
    metric="fscore", 
    direction="max",
)
weather_ranking_model


Model(name: 'weather_ranking_model', version: 1)

In [85]:
no_weather_ranking_model = mr.get_best_model(
    name="no_weather_ranking_model", 
    metric="fscore", 
    direction="max",
)
no_weather_ranking_model

Model(name: 'no_weather_ranking_model', version: 1)

: 

In [ ]:
%%writefile ranking_transformer.py

import os
import pandas as pd

import hopsworks
from opensearchpy import OpenSearch

import logging


class Transformer(object):
    
    def __init__(self):
        # Connect to Hopsworks
        project = hopsworks.connection().get_project()
        self.fs = project.get_feature_store()
        
        # Retrieve the 'videos' feature view
        self.videos_fv = self.fs.get_feature_view(
            name="videos", 
            version=1,
        )
        
        # Get list of feature names for videos
        self.video_features = [feat.name for feat in self.videos_fv.schema]
        
        # Retrieve the 'users' feature view
        self.users_fv = self.fs.get_feature_view(
            name="users", 
            version=1,
        )

        # Retrieve the 'candidate_embeddings' feature view
        self.candidate_index = self.fs.get_feature_view(
            name="candidate_embeddings", 
            version=1,
        )

        # Retrieve ranking model
        mr = project.get_model_registry()
        model = mr.get_model(
            name="ranking_model", 
            version=1,
        )
        
        # Extract input schema from the model
        input_schema = model.model_schema["input_schema"]["columnar_schema"]
        
        # Get the names of features expected by the ranking model
        self.ranking_model_feature_names = [feat["name"] for feat in input_schema]
            
    def preprocess(self, inputs):
        # Extract the input instance
        inputs = inputs["instances"][0]

        # Extract customer_id from inputs
        user_id = inputs["user_id"]
        
        # Search for candidate items
        neighbors = self.candidate_index.find_neighbors(
            inputs["query_emb"], 
            k=100,
        )
        neighbors = [neighbor[0] for neighbor in neighbors]
        
        # Get IDs of videos already seen  by the customer - should filter to only today's videos
        already_seen_videos_ids = self.fs.sql(
            f"SELECT video_id from interactions_1 WHERE user_id = '{user_id}'", online=True
        ).values.reshape(-1).tolist()
        
        # Filter candidate videos to exclude those already seen by the user
        video_id_list = [
            video_id
            for video_id 
            in neighbors 
            if video_id
            not in already_seen_videos_ids
        ]
        video_id_df = pd.DataFrame({"video_id" : video_id_list})
        
        # Retrieve video data for candidate videos
        videos_data = [
            self.videos_fv.get_feature_vector({"video_id": video_id}) 
            for video_id 
            in video_id_list
        ]

        videos_df = pd.DataFrame(
            data=videos_data, 
            columns=self.video_features,
        )
        
        # Join candidate items with their features
        ranking_model_inputs = video_id_df.merge(
            videos_df, 
            on="video_id", 
            how="inner",
        )        
        
        # Add customer features
        user_features = self.users_fv.get_feature_vector(
            {"user_id": user_id}, 
            return_type="pandas",
        )
        
        ranking_model_inputs["user_id"] = user_features['age'].values[0]   
        ranking_model_inputs["gender"] = user_features["gender"].values[0] 
        ranking_model_inputs["age"] = user_features["age"].values[0] 
        ranking_model_inputs["country"] = user_features["country"].values[0] 
        
        # Select only the features required by the ranking model
        ranking_model_inputs = ranking_model_inputs[self.ranking_model_feature_names]
                
        return { 
            "inputs" : [{"ranking_features": ranking_model_inputs.values.tolist(), "video_ids": video_id_list}]
        }

    def postprocess(self, outputs):
        # Extract predictions from the outputs
        preds = outputs["predictions"]
        
        # Merge prediction scores and corresponding article IDs into a list of tuples
        ranking = list(zip(preds["scores"], preds["video_ids"]))
        
        # Sort the ranking list by score in descending order
        ranking.sort(reverse=True)
        
        # Return the sorted ranking list
        return { 
            "ranking": ranking,
        }